# Quantifiably measure the preformance of the greedy heuristic algorithm

In [1]:
import os
import re
import sys

import matplotlib.pyplot as plt
import numpy as np

from itertools import permutations
from math import factorial, log
from time import time

from xmlparse import loadRef, loadGeometryBases, getXmlScore, minXml, loadScores
from score_strokes import alignStrokes, greedyAlign2
from exhaustive import computeExhaustive, exhaustScore

2023-09-26T17:27:25.156822Z [INFO ] Stylus initialized - Stylus 1.5.0 [RELEASE - Aug 29 2023 15:40:46] (c) 2006-2009 Biologic Institute


In [2]:
dlen = 20
f_read = [f"4EFB.2.{i}.gene" for i in range(1, dlen)]

han_char = "4EFB"

ref_g, ref_l, output_size = loadRef(han_char, "Reference")
g_data, _, base_data, stroke_sets, stroke_orders, _ = loadGeometryBases("Genes/maint_0.2 on 4EFB.2", han_char, output_size, f_read = f_read)

good_characters = range(len(g_data))
        
character_num = 14#good_characters[2]
print(f"Testing character {character_num}")

g, l = g_data[character_num]
bases = base_data[character_num]
stroke_set = stroke_sets[character_num]
stroke_order = stroke_orders[character_num]


Testing character 14


In [3]:
# reference-gene alignments are flipped along the index and value - reversing it

heuristic_alignments = greedyAlign2(g, ref_g, l, ref_l)+1

print(heuristic_alignments, stroke_order)


[1 2 6 5 3 4] [1 2 6 5 3 4]


In [4]:
heuristic_xml = minXml(han_char, bases, stroke_set, heuristic_alignments)
original_xml = minXml(han_char, bases, stroke_set, stroke_order)
heuristic_score = getXmlScore(heuristic_xml)
original_score = getXmlScore(original_xml)

In [5]:
for c in good_characters:
    g, l = g_data[c]
    bases = base_data[c]
    stroke_set = stroke_sets[c]
    stroke_order = stroke_orders[c]
    heuristic_alignments = np.array(alignStrokes(g, ref_g, l, ref_l))+1
    heuristic_xml = minXml(han_char, bases, stroke_set, heuristic_alignments)
    original_xml = minXml(han_char, bases, stroke_set, stroke_order)
    heuristic_score = getXmlScore(heuristic_xml)
    original_score = getXmlScore(original_xml)
    print(f"{c}: {heuristic_score}, {original_score}")

0: 0.2009721093859917, 0.2009721093859917
1: 0.2013953338277964, 0.2013953338277964
2: 0.2078045801000123, 0.2078045801000123
3: 0.2016256382450882, 0.2016256382450882
4: 0.2145956745779752, 0.2145956745779752
5: 0.2089713007412017, 0.2089713007412017
6: 0.2034092049312096, 0.2034092049312096
7: 0.2230345742767507, 0.2230345742767507
8: 0.2049115164682263, 0.2049115164682263
9: 0.2086181203229363, 0.2086181203229363
10: 0.2125022264934223, 0.2125022264934223
11: 0.207720919891055, 0.207720919891055
12: 0.2107049646295421, 0.2107049646295421
13: 0.2124287296045458, 0.2124287296045458
14: 0.206119197003043, 0.206119197003043
15: 0.2272769228518867, 0.2272769228518867
16: 0.2091404784088746, 0.2091404784088746
17: 0.207123481074788, 0.207123481074788
18: 0.2196481313045786, 0.2196481313045786


In [6]:
def matchError(heuristic, ref_char, char_data, data_dir="HanBitmap", exhaustive=True, timed=False):
    """
    Inputs:
    heuristic: function to test
    han_char: arctype to test against
    char_data: mutated character data to test

    Error is calculated as log(original_score/heuristic_score, 2) (log base 2)
    This demonstrates the difference in magnitude between the scores,
    which in this context is more relevant that absolute differene
    The log is neccesary to keep numbers from growing obscenely large
    """
    # load in reference geometry
    ref_g, ref_l, output_size = loadRef(ref_char, "Reference")
    # parse character data
    g_data, han_chars, base_data, stroke_sets, stroke_orders, f_names = char_data
    errors = []
    times_heuristic = []
    times_exhaustive = []
    time_store = 0.0
    error_sum = 0.0
    error_max = 0.0
    error_min = np.inf
    exhaustive_scores = []
    for (gl, han_char, bases, stroke_set, stroke_order, f_name) in zip(g_data, han_chars, base_data, stroke_sets, stroke_orders, f_names):
        g, l = gl
        heuristic_alignments = np.array(alignStrokes(g, ref_g, l, ref_l))+1
        heuristic_xml = minXml(han_char, bases, stroke_set, heuristic_alignments)
        # time for heuristic - stored in times_heuristic
        time_store = time()
        heuristic_score = getXmlScore(heuristic_xml)
        times_heuristic.append(time()-time_store)
        if exhaustive:
            # time for exhaustive, stored in times_exhaustive
            if timed:
                time_store = time()
                original_score = exhaustScore(ref_char, han_char, f_name, data_dir, force_refresh = True, save = False)
                times_exhaustive.append(time()-time_store)
            else:
                original_score = exhaustScore(ref_char, han_char, f_name, data_dir)
            exhaustive_scores.append(original_score)
        else:
            original_xml = minXml(han_char, bases, stroke_set, stroke_order)
            original_score = getXmlScore(original_xml)
        error = (heuristic_score)#log(original_score/heuristic_score, 2)
        errors.append(error)
        error_sum += error
        if error > error_max:
            error_max = error
        if error < error_min:
            error_min = error
    error_avg = error_sum/len(g_data)
    print(f"Max Error for pair: {error_max}\nMin Error for set: {error_min}\nAvg Error for set: {error_avg}")
    if timed:
        return errors, times_heuristic, times_exhaustive, exhaustive_scores
    else:
        return error_avg


In [7]:

dlen = 20
f_read = [f"4EFB.2.{i}.gene" for i in range(1, dlen)]

heuristic = alignStrokes
han_char = "4EFB"
data_dir = "Genes/maint_0.2 on 4EFB.2"
char_data = loadGeometryBases(data_dir, han_char, output_size)#, f_read = f_read)

matchError(heuristic, han_char, char_data, data_dir)


Max Error for pair: 0.2272769228518867
Min Error for set: 0.2009721093859917
Avg Error for set: 0.20957448259465875


0.20957448259465875